### Parâmetros do modelo:

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from random import randint
from sklearn.manifold import TSNE
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# Evitar warnings:
import warnings
warnings.filterwarnings("ignore")

In [3]:
CUT_OFF_DATE = pd.to_datetime('2022-03-31')
DATE_OF_INFERENCE = '2023-11-17'

In [4]:
def criar_df_atemporal(dt):
    # Substitua o caminho das bases aqui:
    subscriptions_features_dataset_filepath = '../Tabelas/subscriptions_features_dataset.csv'
    features_filepath = '../Tabelas/features.csv'

    x1 = pd.read_csv(subscriptions_features_dataset_filepath)
    x5 = pd.read_csv(features_filepath)

    df_final = pd.merge(x1, x5, on='employee_id', how='left')

    # Selecionar colunas para clusterização
    col_cluster = ['employee_id','type_mapped', 'gender_maped', 
                'Binario_fee', 'Fee', 'NumGymsWithinRadius', 
                'NumGymsNearCompany', 'distancia_cliente_empresa',
                'payment_source_y']
    df_atemp = df_final[col_cluster]
    df_atemp['employee_id'] = df_atemp['employee_id'].astype(int) 

    return df_atemp

In [25]:
def criar_df_temporal(CUT_OFF_DATE,dt):
    def calcular_gym_status(CUT_OFF_DATE,df):
        today = CUT_OFF_DATE
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        df = df.drop('Unnamed: 0', axis=1)
        df1 = df[df['validated_at'] <= today]
        df2 = df1[['validated_at','employee_id','token','gym_id']]
        academias_mais_utilizadas = df2.groupby(['employee_id', 'gym_id']).size().reset_index(name='utilizacoes')
        idx = academias_mais_utilizadas.groupby(['employee_id'])['utilizacoes'].transform(max) == academias_mais_utilizadas['utilizacoes']
        academias_mais_utilizadas = academias_mais_utilizadas[idx]
        gyms = pd.read_csv('C:\\Users\\Pedro Nakandakari\\Downloads\\Total Pass\\gyms.csv')
        gyms = gyms.drop('Unnamed: 0', axis=1)
        gyms['registered_at'] = pd.to_datetime(gyms['registered_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['activated_at'] = pd.to_datetime(gyms['activated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['last_inactivated_at'] = pd.to_datetime(gyms['last_inactivated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms1 = gyms[gyms['registered_at'] <= today]
        gyms2 = gyms1[['gym_id','gym_status']]
        dfprinc = pd.merge(gyms2, academias_mais_utilizadas, on='gym_id', how='right')
        return dfprinc
    def calcular_age(CUT_OFF_DATE):
        df2 = pd.read_csv('../Tabelas/employees.csv')

        df2['birthdate'] = pd.to_datetime(df2['birthdate'], errors='coerce')
        df2 = df2.dropna(subset=['birthdate'])

        today = pd.to_datetime(CUT_OFF_DATE)

        # iterando o df2 calculando a coluna 'age'
        df2['age'] = df2.apply(lambda row: today.year - row['birthdate'].year - ((today.month < row['birthdate'].month) or ((today.month == row['birthdate'].month) and (today.day < row['birthdate'].day))), axis=1)

        df2 = df2[['employee_id','age']]
        return df2
    def calcular_freezes(CUT_OFF_DATE):
        df1 = pd.read_csv('../Tabelas/freezes.csv')
        df2 = pd.read_csv('../Tabelas/subscriptions.csv')


        distinct_ids_counts = df1.groupby('employee_id')['id'].nunique().reset_index()
        distinct_ids_counts.columns = ['employee_id', 'freezes']

        df2 = pd.merge(df2, distinct_ids_counts, on='employee_id', how='left')
        df2['freezes'].fillna(0, inplace=True)

        df2 = df2[['employee_id', 'freezes']]
        
        return df2
    def calcular_upgrade(CUT_OFF_DATE):
        df2 = pd.read_csv('../Tabelas/subscriptions.csv')

        df2['upgrade'] = np.where(df2['migrated_from_subscription_id'].notna(), 1, 0)

        filtered_subscriptions = df2[df2['upgrade'] == 0]

        count_subscriptions_by_employee = filtered_subscriptions.groupby('employee_id')['subscription_id'].count().reset_index()

        count_subscriptions_by_employee = count_subscriptions_by_employee.rename(columns={'subscription_id': 'N° Subscriptions'})

        df2 = pd.merge(df2, count_subscriptions_by_employee, on='employee_id', how='left')

        df2 = df2[['employee_id', 'upgrade','N° Subscriptions']]

        return df2
    def calcular_pref_modality_utilizaçoes(CUT_OFF_DATE,df):
        today = pd.to_datetime(CUT_OFF_DATE)
        df['validated_at'] = pd.to_datetime(df['validated_at'])
        df['created_at'] = pd.to_datetime(df['created_at'])
        df = df.drop('Unnamed: 0', axis=1)
        df1 = df[df['validated_at'] <= today]
        gyms = pd.read_csv('C:\\Users\\Pedro Nakandakari\\Downloads\\Total Pass\\gyms.csv')
        gyms = gyms.drop('Unnamed: 0', axis=1)
        gyms['registered_at'] = pd.to_datetime(gyms['registered_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['activated_at'] = pd.to_datetime(gyms['activated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms['last_inactivated_at'] = pd.to_datetime(gyms['last_inactivated_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        gyms1 = gyms[gyms['registered_at'] <= today]
        df2 = df1[['employee_id','gym_id']]
        gyms2 = gyms1[['gym_id','main_modality']]
        gymsdf = pd.merge(gyms2, df2, how= 'right', on= ['gym_id'])
        modalidades_preferidas = gymsdf.groupby(['employee_id', 'main_modality']).size().reset_index(name='num_main_modality')
        idx1 = modalidades_preferidas.groupby(['employee_id'])['num_main_modality'].transform(max) == modalidades_preferidas['num_main_modality']
        modalidades_preferidas = modalidades_preferidas[idx1]
        modalidades_preferidas = modalidades_preferidas.rename(columns={'main_modality': 'pref_modality'})
        gymsprinc = pd.merge(gymsdf, modalidades_preferidas[['employee_id', 'pref_modality', 'num_main_modality']],
                on=['employee_id'], how='left')
        return gymsprinc
    def calcular_num_distinct_gyms(CUT_OFF_DATE,df):
        today = pd.to_datetime(CUT_OFF_DATE)
        df1 = df[df['validated_at'] <= today]
        df1['validated_at'] = pd.to_datetime(df1['validated_at'])
        df1['created_at'] = pd.to_datetime(df1['created_at'])
        df1 = df1.drop('Unnamed: 0', axis=1)
        df2 = df1[['employee_id','gym_id']]
        academias_distintas_por_usuario = df2.groupby('employee_id')['gym_id'].nunique().reset_index(name='num_distinct_gyms')
        distpref = pd.merge(df2, academias_distintas_por_usuario, on='employee_id', how='left')
        return distpref
    def calcular_uses_per_week(CUT_OFF_DATE,df):
        df['validated_at_minute'] = pd.to_datetime(df['validated_at_minute'])
        df1 = df[['employee_id','validated_at_minute']]    
        df1['Ano'] = df1['validated_at_minute'].dt.year
        df1['Semana'] = df1['validated_at_minute'].dt.isocalendar().week
        today = pd.to_datetime(CUT_OFF_DATE)
        df2 = df1[df1['validated_at_minute'] <= today]
        num_utilizacoes_por_semana = df1.groupby(['employee_id', 'Ano', 'Semana']).size().reset_index(name='Num_Utilizacoes')
        media_utilizacoes_por_semana = num_utilizacoes_por_semana.groupby('employee_id')['Num_Utilizacoes'].mean().reset_index(name='uses_per_week')
        df3 = pd.merge(df2, media_utilizacoes_por_semana, how= 'left', on= ['employee_id'])
        return df3
    def calcular_most_frequent_hour(CUT_OFF_DATE,df):
        df['validated_at_minute'] = pd.to_datetime(df['validated_at_minute'])
        df1 = df[['employee_id','validated_at_minute']]
        today = pd.to_datetime(CUT_OFF_DATE)
        df2 = df1[df1['validated_at_minute'] <= today]
        hora_mais_frequente_por_usuario = df2.groupby('employee_id')['validated_at_minute'].agg(lambda x: x.dt.hour.mode().iloc[0])
        df3 = pd.merge(df2, hora_mais_frequente_por_usuario, on='employee_id', how='left')
        return df3
    def active_at_date(CUT_OFF_DATE):
        df2 = pd.read_csv('../Tabelas/subscriptions.csv')

        df2['created_at'] = pd.to_datetime(df2['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        df2['canceled_at'] = pd.to_datetime(df2['canceled_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        
        CUT_OFF_DATE = pd.to_datetime(CUT_OFF_DATE)

        # Selecionando 1 apenas as assinaturas ativas na data de corte
        df2['active_at_date'] = ((df2['created_at'] <= CUT_OFF_DATE) & ((df2['canceled_at'] >= CUT_OFF_DATE) | (df2['canceled_at'].isnull()))).astype(int)
        # Para ser ativo,
        # 1. A assinatura deve ter sido criada antes da data de corte
        # 2. A assinatura deve ter sido cancelada depois da data de corte ou não ter sido cancelada

        df2 = df2[['employee_id','active_at_date']]
        return df2
    def churn_at_dt(CUT_OFF_DATE,dt):
        df2 = pd.read_csv('../Tabelas/subscriptions.csv')

        CUT_OFF_DATE = pd.to_datetime(CUT_OFF_DATE)

        df2['created_at'] = pd.to_datetime(df2['created_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        df2['canceled_at'] = pd.to_datetime(df2['canceled_at'], format="%Y-%m-%dT%H:%M:%S.%fZ")
        print(CUT_OFF_DATE+pd.Timedelta(dt))
        
        # Selecionando 1 apenas as assinaturas ativas na data de corte
        df2['churn_at_dt'] = ((df2['created_at'] <= CUT_OFF_DATE) & ((df2['canceled_at'] >= CUT_OFF_DATE) & (df2['canceled_at'] <= CUT_OFF_DATE + pd.Timedelta(days = dt)))).astype(int)
        # Para dar churn no gap,
        # 1. A assinatura deve ter sido criada antes da data de corte

        df2 = df2[['employee_id','churn_at_dt']]
        return df2
    
    # Carregando todas as funções com CUT_OFF_DATE
    df_age = calcular_age(CUT_OFF_DATE)
    print('Calculou age')
    df_freezes = calcular_freezes(CUT_OFF_DATE)
    print('Calculou freezes')
    df_upgrade = calcular_upgrade(CUT_OFF_DATE)
    print('Calculou upgrade')

    # df_token = pd.read_csv('../Tabelas/daily_token_validation-001.csv')
    # df_gym_status = calcular_gym_status(CUT_OFF_DATE,df_token)
    # df_pref_modality = calcular_pref_modality_utilizaçoes(CUT_OFF_DATE,df_token)
    # df_num_distinct_gyms = calcular_num_distinct_gyms(CUT_OFF_DATE,df_token)
    # df_uses_per_week = calcular_uses_per_week(CUT_OFF_DATE,df_token)
    # df_most_frequent_hour = calcular_most_frequent_hour(CUT_OFF_DATE,df_token)

    df_active_at_date = active_at_date(CUT_OFF_DATE)
    print('Calculou active_at_date')
    df_churn_at_dt = churn_at_dt(CUT_OFF_DATE, dt)
    print('Calculou churn_at_dt')

    # Merge nos 3 DataFrames com chave 'employee_id'
    df_temp = pd.merge(df_age, df_upgrade, on='employee_id', how='inner')
    print("Após o primeiro merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_freezes, on='employee_id', how='inner')
    print("Após o segundo merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_active_at_date, on='employee_id', how='inner')
    print("Após o terceiro merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    print("Após remover duplicatas:", df_temp.shape)

    df_temp = pd.merge(df_temp, df_churn_at_dt, on='employee_id', how='right')
    print("Após o quarto merge:", df_temp.shape)

    # Remove duplicatas:
    df_temp = df_temp.drop_duplicates(subset='employee_id', keep='first')
    print("Após remover duplicatas:", df_temp.shape)

    return df_temp


In [6]:
def merge_dfs(df_atemp, df_temp):
    df = pd.merge(df_atemp, df_temp, on='employee_id', how='left')
    print("Após o merge:", df.shape)

    # Remove duplicatas:
    df = df.drop_duplicates(subset='employee_id', keep='first')
    print("Após remover duplicatas:", df.shape)

    display(df)

    return df

In [7]:
def preprocessa_df(df):
    # dropar nulos com base na coluna 'active_at_date'
    df = df.dropna(subset=['active_at_date'])

    # Substitua active por 1 e inactive por 0 em gym status:
    #df['gym_status'] = df['gym_status'].map({'active': 1, 'inactive': 0})
    # Substitua musculação por 1 e qualquer outra categoria por 0 em pref modality:
    #df['pref_modality'] = df['pref_modality'].map(lambda x: 1 if x == 'MUSCULAÇÃO' else 0)
    # Substitua payroll por 0, prepaid por 1 e own_wallet por 2 em payment source:
    df['payment_source_y'] = df['payment_source_y'].map({'payroll_deduction': 0, 'prepaid': 1, 'own_wallet': 2,})

    # Preenchendo valores nulos com 0
    df['Binario_fee'].fillna(0, inplace=True)
    df['Fee'].fillna(0, inplace=True)
    df['freezes'].fillna(0, inplace=True)
    df['upgrade'].fillna(0, inplace=True)
    df['N° Subscriptions'].fillna(0, inplace=True)
    df['payment_source_y'].fillna(0, inplace=True)
    #df['pref_modality'].fillna(1, inplace=True) # Preenche com musculação

    # Preenchendo valores nulos com a média
    media_distancia = df['distancia_cliente_empresa'].mean()
    media_idade = df['age'].mean()
    media_genero = df['gender_maped'].mean()

    # Preenche os valores NaN com as médias calculadas
    df['distancia_cliente_empresa'].fillna(media_distancia, inplace=True)
    df['age'].fillna(media_idade, inplace=True)
    df['gender_maped'].fillna(media_genero, inplace=True)
    df.dropna(subset=['type_mapped', 'NumGymsWithinRadius'], inplace=True)

    df = df[df['active_at_date'] == 1]

    return df

In [8]:
def treinar_modelo(df, dt):
    # Definindo as variáveis preditoras e a variável resposta
    X = df.drop(['employee_id', 'churn_at_dt','active_at_date'], axis=1)
    # Normalizando os dados
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    y = df['churn_at_dt']

    # Dividindo os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Instanciando o modelo de nome
    logreg = LogisticRegression()

    # Treinando o modelo
    logreg.fit(X_train, y_train)

    # Fazendo previsões
    y_pred = logreg.predict(X_test)

    # Avaliando o modelo
    print('Acurácia:', accuracy_score(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

    return logreg

In [26]:
def previsao(dt, X_inferencia):
    df_atemp = criar_df_atemporal(dt)
    print('DF Atemporal criado')
    df_temp = criar_df_temporal(CUT_OFF_DATE,dt)
    print('DF Temporal criado')
    df = merge_dfs(df_atemp, df_temp)
    print('DF Mergeado')
    df = preprocessa_df(df)
    print('DF Preprocessado')
    model = treinar_modelo(df, dt)
    print('Modelo treinado')
    previsao = model.predict_proba(X_inferencia)
    return previsao

In [27]:
AGE = 30
UPGRADE = 0
FREEZES = 0
N_SUBSCRIPTIONS = 1
TYPE_MAPPED = 0
GENDER_MAPED = 1
BINARIO_FEE = 0
FEE = 1
NUM_GYM_WITHIN_RADIUS = 0
NUM_GYM_NEAR_COMPANY = 0
DISTANCIA_CLIENTE_EMPRESA = 100
PAYMENT_SOURCE_Y = 2

# Agrupe todas as variáveis
X_inferencia = [[AGE, UPGRADE, FREEZES, N_SUBSCRIPTIONS, TYPE_MAPPED, GENDER_MAPED, BINARIO_FEE, FEE, NUM_GYM_WITHIN_RADIUS, NUM_GYM_NEAR_COMPANY, DISTANCIA_CLIENTE_EMPRESA, PAYMENT_SOURCE_Y]]

scaler = MinMaxScaler()

# Normalizando os dados
X_inferencia = scaler.fit_transform(X_inferencia)

for dt in [90,180,270]:
    inferencia = previsao(dt, X_inferencia)
    print(f'Previsão de churn em {dt} dias:', inferencia)

DF Atemporal criado
Calculou age
Calculou freezes
Calculou upgrade
Calculou active_at_date
2022-03-31 00:00:00.000000090
Calculou churn_at_dt
Após o primeiro merge: (1558762, 4)
Após remover duplicatas: (1099154, 4)
Após o segundo merge: (1558762, 5)
Após remover duplicatas: (1099154, 5)
Após o terceiro merge: (1558762, 6)
Após remover duplicatas: (1099154, 6)
Após o quarto merge: (1605146, 7)
Após remover duplicatas: (1136775, 7)
DF Temporal criado
Após o merge: (1764656, 15)
Após remover duplicatas: (1136775, 15)


,employee_id,type_mapped,gender_maped,Binario_fee,Fee,NumGymsWithinRadius,NumGymsNearCompany,distancia_cliente_empresa,payment_source_y,age,upgrade,N° Subscriptions,freezes,active_at_date,churn_at_dt
0,130,0.0,NaN,0.0,0.0,211.0,505,12.876877,NaN,36.0,0.0,2.0,0.0,0.0,0
1,162,0.0,NaN,1.0,420.0,309.0,505,5.561663,NaN,31.0,0.0,1.0,0.0,0.0,0
2,28163,0.0,NaN,0.0,0.0,245.0,217,2.580917,NaN,46.0,0.0,1.0,0.0,0.0,0
3,21739,0.0,NaN,0.0,0.0,14.0,217,874.545486,NaN,34.0,0.0,1.0,0.0,0.0,0
4,93992,0.0,NaN,1.0,300.0,537.0,217,8.854492,NaN,36.0,0.0,2.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764648,2091820,1.0,1.0,NaN,NaN,224.0,0,NaN,NaN,35.0,0.0,1.0,0.0,0.0,0
1764649,2091851,1.0,1.0,NaN,NaN,131.0,353,18.229972,NaN,28.0,0.0,1.0,0.0,0.0,0
1764652,2006364,1.0,0.0,NaN,NaN,177.0,500,9.818316,NaN,37.0,0.0,1.0,0.0,0.0,0
1764654,2004863,1.0,1.0,NaN,NaN,185.0,500,9.887746,NaN,37.0,0.0,1.0,0.0,0.0,0


DF Mergeado
DF Preprocessado
Acurácia: 0.7140558898885777
[[23734   240]
 [ 9358   234]]
              precision    recall  f1-score   support

           0       0.72      0.99      0.83     23974
           1       0.49      0.02      0.05      9592

    accuracy                           0.71     33566
   macro avg       0.61      0.51      0.44     33566
weighted avg       0.65      0.71      0.61     33566

Modelo treinado
Previsão de churn em 90 dias: [[0.20552689 0.79447311]]
DF Atemporal criado
Calculou age
Calculou freezes
Calculou upgrade
Calculou active_at_date
2022-03-31 00:00:00.000000180
Calculou churn_at_dt
Após o primeiro merge: (1558762, 4)
Após remover duplicatas: (1099154, 4)
Após o segundo merge: (1558762, 5)
Após remover duplicatas: (1099154, 5)
Após o terceiro merge: (1558762, 6)
Após remover duplicatas: (1099154, 6)
Após o quarto merge: (1605146, 7)
Após remover duplicatas: (1136775, 7)
DF Temporal criado
Após o merge: (1764656, 15)
Após remover duplicatas: (1136

,employee_id,type_mapped,gender_maped,Binario_fee,Fee,NumGymsWithinRadius,NumGymsNearCompany,distancia_cliente_empresa,payment_source_y,age,upgrade,N° Subscriptions,freezes,active_at_date,churn_at_dt
0,130,0.0,NaN,0.0,0.0,211.0,505,12.876877,NaN,36.0,0.0,2.0,0.0,0.0,0
1,162,0.0,NaN,1.0,420.0,309.0,505,5.561663,NaN,31.0,0.0,1.0,0.0,0.0,0
2,28163,0.0,NaN,0.0,0.0,245.0,217,2.580917,NaN,46.0,0.0,1.0,0.0,0.0,0
3,21739,0.0,NaN,0.0,0.0,14.0,217,874.545486,NaN,34.0,0.0,1.0,0.0,0.0,0
4,93992,0.0,NaN,1.0,300.0,537.0,217,8.854492,NaN,36.0,0.0,2.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764648,2091820,1.0,1.0,NaN,NaN,224.0,0,NaN,NaN,35.0,0.0,1.0,0.0,0.0,0
1764649,2091851,1.0,1.0,NaN,NaN,131.0,353,18.229972,NaN,28.0,0.0,1.0,0.0,0.0,0
1764652,2006364,1.0,0.0,NaN,NaN,177.0,500,9.818316,NaN,37.0,0.0,1.0,0.0,0.0,0
1764654,2004863,1.0,1.0,NaN,NaN,185.0,500,9.887746,NaN,37.0,0.0,1.0,0.0,0.0,0


DF Mergeado
DF Preprocessado
Acurácia: 0.5839540010725138
[[14860  3778]
 [10187  4741]]
              precision    recall  f1-score   support

           0       0.59      0.80      0.68     18638
           1       0.56      0.32      0.40     14928

    accuracy                           0.58     33566
   macro avg       0.57      0.56      0.54     33566
weighted avg       0.58      0.58      0.56     33566

Modelo treinado
Previsão de churn em 180 dias: [[0.07024727 0.92975273]]
DF Atemporal criado
Calculou age
Calculou freezes
Calculou upgrade
Calculou active_at_date
2022-03-31 00:00:00.000000270
Calculou churn_at_dt
Após o primeiro merge: (1558762, 4)
Após remover duplicatas: (1099154, 4)
Após o segundo merge: (1558762, 5)
Após remover duplicatas: (1099154, 5)
Após o terceiro merge: (1558762, 6)
Após remover duplicatas: (1099154, 6)
Após o quarto merge: (1605146, 7)
Após remover duplicatas: (1136775, 7)
DF Temporal criado
Após o merge: (1764656, 15)
Após remover duplicatas: (113

,employee_id,type_mapped,gender_maped,Binario_fee,Fee,NumGymsWithinRadius,NumGymsNearCompany,distancia_cliente_empresa,payment_source_y,age,upgrade,N° Subscriptions,freezes,active_at_date,churn_at_dt
0,130,0.0,NaN,0.0,0.0,211.0,505,12.876877,NaN,36.0,0.0,2.0,0.0,0.0,0
1,162,0.0,NaN,1.0,420.0,309.0,505,5.561663,NaN,31.0,0.0,1.0,0.0,0.0,0
2,28163,0.0,NaN,0.0,0.0,245.0,217,2.580917,NaN,46.0,0.0,1.0,0.0,0.0,0
3,21739,0.0,NaN,0.0,0.0,14.0,217,874.545486,NaN,34.0,0.0,1.0,0.0,0.0,0
4,93992,0.0,NaN,1.0,300.0,537.0,217,8.854492,NaN,36.0,0.0,2.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1764648,2091820,1.0,1.0,NaN,NaN,224.0,0,NaN,NaN,35.0,0.0,1.0,0.0,0.0,0
1764649,2091851,1.0,1.0,NaN,NaN,131.0,353,18.229972,NaN,28.0,0.0,1.0,0.0,0.0,0
1764652,2006364,1.0,0.0,NaN,NaN,177.0,500,9.818316,NaN,37.0,0.0,1.0,0.0,0.0,0
1764654,2004863,1.0,1.0,NaN,NaN,185.0,500,9.887746,NaN,37.0,0.0,1.0,0.0,0.0,0


DF Mergeado
DF Preprocessado
Acurácia: 0.5846988023595304
[[ 5472  9825]
 [ 4115 14154]]
              precision    recall  f1-score   support

           0       0.57      0.36      0.44     15297
           1       0.59      0.77      0.67     18269

    accuracy                           0.58     33566
   macro avg       0.58      0.57      0.55     33566
weighted avg       0.58      0.58      0.57     33566

Modelo treinado
Previsão de churn em 270 dias: [[0.04221532 0.95778468]]
